In [3]:
#Imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, confusion_matrix, precision_score, accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from Utilities import utils

## Data Processsing

In [2]:
# Load the raw MAFLD data.
data_dir = "Data/NTCMRC_all.xlsx"
df = pd.read_excel(data_dir)

In [3]:
# Create a copy of df as df1
df1 = df.copy()

# Replace '\\N' with NaN
df1 = df1.replace('\\N', np.nan)

# Specify the columns to be converted to FLOAT
columns_to_convert1 = ['BMI', 'Triglyceride_y', 'gamgt', 'waist_y', 'mst', 'egfrn', 'Estimated_GFR_x', 'Alb_Cre_ratio', 'HOMA_IR', 'HS_CRP', \
                       'LDL_C_direct', 'LDL_C_HDL_C', 'Adiponectin', 'Leptin', 'Uric_Acid','Insulin', 'ALT_GPT']

# Specify the columns to be converted to INT
columns_to_convert2 = ['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)', 'smoke', 'smoke_q', \
                       'sex', 'w', 'coffee', 'betel']

# Convert the specified columns to float and fill missing/unconvertible values with NaN
for column in columns_to_convert1:
    df1[column] = pd.to_numeric(df1[column], errors='coerce')

# Convert the specified columns to int and fill missing/unconvertible values with NaN
for column in columns_to_convert2:
    df1[column] = pd.to_numeric(df1[column], errors='coerce').astype(pd.Int64Dtype())

# Calculate FLI using the formula and defined as df2
df2 = df1.copy()
df2['FLI'] = (np.exp(0.953 * np.log(df2['Triglyceride_y']) + 0.139 * df2['BMI'] + 0.718 * np.log(df2['gamgt']) \
     + 0.053 * df2['waist_y'] - 15.745)) / (1 + np.exp(0.953 * np.log(df2['Triglyceride_y']) \
    + 0.139 * df2['BMI'] + 0.718 * np.log(df2['gamgt']) + 0.053 * df2['waist_y'] - 15.745)) * 100

# Derive FL_echo based on ultrasound results column
df2['FL_echo'] = df2['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)']
df2['FL_echo'] = df2['FL_echo'].replace('<NA>', np.nan)
df2['fl_status'] = df2.apply(utils.derive_fl_status, axis=1)

#Derive homa_ir_check, hs_crp_check, and mst_total to determine MAFLD risk factors
df2['homa_ir_check'] = df2['HOMA_IR'].apply(lambda x: 1 if x >= 2.5 else 0)
df2['hs_crp_check'] = df2['HS_CRP'].apply(lambda x: 1 if x > 2 else 0)
df2['mst_total'] = df2[['w', 'hyper', 'HDL', 'fg', 'trig', 'homa_ir_check', 'hs_crp_check']].sum(axis=1)

df3 = utils.derive_MAFLD_with_multi_label(utils.derive_MAFLD(df2))
df4 = utils.derive_CKD(df3)

columns_to_extract = ['HBsAg_x', 'Anti_HCV_x']
df5 = df4.copy()
for column in columns_to_extract:
    new_column_name = column + '_num'
    df5[new_column_name] = df5[column].apply(utils.extract_numeric_value)

# Filtering - for patients with same CMRC)id and year_come has more than two records
count_2_or_more_year_come = df5.groupby(['CMRC_id', 'year_come']).filter(lambda x: len(x) >= 2)
unique_patients = count_2_or_more_year_come['CMRC_id'].nunique()
patients_to_remove = df5.groupby(['CMRC_id', 'year_come']).filter(lambda x: len(x) >= 2)['CMRC_id'].unique()
df5_filtered = df5[~df5['CMRC_id'].isin(patients_to_remove)]

df6 = utils.sliding_window_multi_label_data(df5_filtered, input_window_size=1, target_window_size=1)

# Filtering - filter df with first year patient with NON-MAFLD and target year(2nd year) MAFLD status valid
filtered_df1 = df6[(df6['t1_MAFLD_0'] == 1) & (df6['t2_MAFLD_0'] != -1)]

In [4]:
# Feature Selection - manually selected by domain expert

# Drop ID relevant cols in the dataset
columns_to_drop = ['CMRC_id','t1_CMRC_id', 't1_sid','t1_P_Number']
df8 = filtered_df1.drop(columns=columns_to_drop)

#Select key columns for conventional machine learning models
columns = ["sex", "age", "waist_y", "Glucose_AC_y", "Triglyceride_y", "HDL_C_y", "AST_GOT", "ALT_GPT", \
          "gamgt", "Insulin", "T_Cholesterol", "LDL_C_direct", "VLDL_C", "Non_HDL_C", "T_CHOL_HDL_C", \
          "LDL_C_HDL_C", "HS_CRP", "Hb_A1c", "Uric_Acid", "HBsAg_x", "Anti_HCV_x", "HOMA_IR", "Adiponectin", \
           "Leptin", "TotalVitaminD", "smoke", "smoke_q", "coffee", "betel", "BMI", "DM_determine", "w", "hyper", \
           "fg", "HDL", "trig", "sarcf", "ms2", "MNA", "AUDIT", "HBV_", "HCV_", "MAFLD", "CKD", \
           'HBsAg_x_num', 'Anti_HCV_x_num', \
           'MAFLD_0', 'MAFLD_Obesity', 'MAFLD_MD', 'MAFLD_Diabetes', \
           'year_come']
# prefixes = ["t1_", "t2_"]
prefixes = ["t1_"]
renamed_columns = utils.add_prefix(columns, prefixes)

df9 = df8[renamed_columns]
# df9['t2_MAFLD'] = df8['t2_MAFLD']
df9['t2_MAFLD_0'] = df8['t2_MAFLD_0']
df9['t2_MAFLD_Obesity'] = df8['t2_MAFLD_Obesity']
df9['t2_MAFLD_MD'] = df8['t2_MAFLD_MD']
df9['t2_MAFLD_Diabetes'] = df8['t2_MAFLD_Diabetes']

# drop these cols as those been derived for numeric cols, remain alias *_num, and MAFLD status for 1 year
cols_to_drop_only_MAFLD = ['t1_HBsAg_x',  't1_Anti_HCV_x', 't1_MAFLD', 't1_MAFLD_0',\
                           't1_MAFLD_Obesity', 't1_MAFLD_MD', 't1_MAFLD_Diabetes', \
                           ]

df9_processed = df9.drop(cols_to_drop_only_MAFLD, axis=1)

C:\Users\zyj\AppData\Local\Temp\ipykernel_4068\2510877545.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df9['t2_MAFLD_0'] = df8['t2_MAFLD_0']
C:\Users\zyj\AppData\Local\Temp\ipykernel_4068\2510877545.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df9['t2_MAFLD_Obesity'] = df8['t2_MAFLD_Obesity']
C:\Users\zyj\AppData\Local\Temp\ipykernel_4068\2510877545.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

## Data set split for train-validation-test

In [5]:
# 前4年 training， 4-5 testing
# filter for for training set
filtered_df_train = df9_processed[df9_processed['t1_year_come'] <= 3]

# filter for year4 for test set
filtered_df_test = df9_processed[df9_processed['t1_year_come'] >= 4]

# df9_processed.drop('t1_MAFLD_0', axis=1, inplace=True)
features = df9_processed.columns.drop(['t2_MAFLD_0', 't2_MAFLD_Obesity', 't2_MAFLD_Diabetes', 't2_MAFLD_MD'])

categorical_features = ['t1_sex', 't1_w', 't1_smoke', 't1_smoke_q', 't1_coffee', 't1_betel', 't1_DM_determine', 't1_CKD']
numeric_features = df9_processed.columns.drop(categorical_features).drop(['t2_MAFLD_0', 't2_MAFLD_Obesity', 't2_MAFLD_Diabetes', 't2_MAFLD_MD'])
X_categorical = df9_processed[categorical_features]
X_numeric = df9_processed[numeric_features]

# multi-label for target variable
y = df9_processed[['t2_MAFLD_0', 't2_MAFLD_Obesity', 't2_MAFLD_Diabetes', 't2_MAFLD_MD']]

# Scaling
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Missing value handling
imputer = SimpleImputer(strategy='median')
X_numeric_scaled_imputed = imputer.fit_transform(X_numeric_scaled)

#dummy var
X_categorical_str = X_categorical.astype(str)
X_categorical_encoded = pd.get_dummies(X_categorical_str, drop_first=True)

# concat
X_numeric_scaled_imputed = pd.DataFrame(X_numeric_scaled_imputed, columns=X_numeric.columns)
X_numeric_scaled_imputed.reset_index(drop=True, inplace=True)
X_categorical_encoded.reset_index(drop=True, inplace=True)
X_combined = pd.concat([X_numeric_scaled_imputed, X_categorical_encoded], axis=1)

In [6]:
# 前4年 training， 4-5 testing
# filter for for training set
df_train = df9_processed[df9_processed['t1_year_come'] <= 3]

# filter for year4 for test set
df_test_and_validation = df9_processed[df9_processed['t1_year_come'] >= 4]

# Separate data with Train, validate, test set
df_validation = df_test_and_validation.sample(n=500, random_state=1)
df_test = df_test_and_validation.drop(df_validation.index)

print(f"The shape of the training set is {df_train.shape}")
print(f"The shape of the validation set is {df_validation.shape}")
print(f"The shape of the testing set is {df_test.shape}")

The shape of the training set is (8671, 48)
The shape of the validation set is (500, 48)
The shape of the testing set is (2027, 48)


In [7]:
# Usage
# feature processing for scaling and missing imputation
X_train, y_train = utils.preprocess_features_and_target(df_train)
X_val, y_val = utils.preprocess_features_and_target(df_validation)
X_test, y_test = utils.preprocess_features_and_target(df_test)

In [8]:
# Convert all dataframes to float
X_train = X_train.astype(float)
y_train = y_train.astype(float)
X_val = X_val.astype(float)
y_val = y_val.astype(float)
X_test = X_test.astype(float)
y_test = y_test.astype(float)

## Experiment and Modeling

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, hamming_loss, f1_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [10]:
# List of target variables - aim to apply 4 classfier for each target label
targets = ['t2_MAFLD_0', 't2_MAFLD_Obesity', 't2_MAFLD_Diabetes', 't2_MAFLD_MD']

### Logistic Regression

In [11]:
# Dictionary to store AUC scores for each target
auc_scores = {}

# Initialize an empty DataFrame for predictions
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

print("---Starting Training Model with Logistic Regression---")
# Train a separate Logistic Regression model for each target
for target in targets:
    # Initialize Logistic Regression and GridSearchCV
    lr = LogisticRegression(solver='liblinear', random_state=2023)
    grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_lr = grid_search.best_estimator_
    best_params = grid_search.best_params_
    print(f"Best hyperparameters for {target}: {best_params}")

    # Predict on the test set
    y_pred_prob = best_lr.predict_proba(X_test)[:, 1]
    
    # Calculate AUC on the test set and store in the dictionary
    auc_score = roc_auc_score(y_test[target], y_pred_prob)
    auc_scores[target] = auc_score

    # Add predictions to the DataFrame
    y_pred_df[target] = best_lr.predict(X_test)

# Calculate accuracy on the test set
print("\n---Apply Best Hyper Param for Test Set---")
accuracy = accuracy_score(y_test[targets], y_pred_df)
print(f"Overall Test Accuracy: {accuracy}")
# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()
ham_loss = hamming_loss(y_test[targets], y_pred_array)
print(f"Hamming Loss: {ham_loss}")
f1_macro = f1_score(y_test[targets], y_pred_array, average='macro')
f1_micro = f1_score(y_test[targets], y_pred_array, average='micro')
print(f"F1 Score (Macro): {f1_macro}")
print(f"F1 Score (Micro): {f1_micro}")

# Print AUC for each target
print("AUC Scores for Each Target:")
for target, score in auc_scores.items():
    print(f"{target}: {score}")

---Starting Training Model with Logistic Regression---
Best hyperparameters for t2_MAFLD_0: {'C': 0.1}
Best hyperparameters for t2_MAFLD_Obesity: {'C': 0.1}
Best hyperparameters for t2_MAFLD_Diabetes: {'C': 1}
Best hyperparameters for t2_MAFLD_MD: {'C': 1}

---Apply Best Hyper Param for Test Set---
Overall Test Accuracy: 0.8776517020226936
Hamming Loss: 0.06401085347804637
F1 Score (Macro): 0.28828413284132837
F1 Score (Micro): 0.8740596942489687
AUC Scores for Each Target:
t2_MAFLD_0: 0.7967054077418115
t2_MAFLD_Obesity: 0.8190598940944455
t2_MAFLD_Diabetes: 0.9156662203981301
t2_MAFLD_MD: 0.7963409519815474


### SVM

In [32]:
# Initialize DataFrames for storing probabilities and predictions
y_prob_df = pd.DataFrame()
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10]}

print("---Starting Training Model with SVM---")
# Loop through each target variable for model training and evaluation
for target in targets:
    # Initialize SVC and GridSearchCV
    svm = SVC(probability=True)
    grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_svm = grid_search.best_estimator_
    print(f"Best hyperparameters for {target}: {grid_search.best_params_}")

    # Predict on the test set
    y_pred = best_svm.predict(X_test)

    # Store the predictions
    y_pred_df[target] = y_pred

# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()

# Calculate overall performance metrics
accuracy = accuracy_score(y_test[targets], y_pred_array)
ham_loss = hamming_loss(y_test[targets], y_pred_array)
f1_scores_macro = [f1_score(y_test[target], y_pred_df[target], average='macro') for target in targets]
f1_scores_micro = [f1_score(y_test[target], y_pred_df[target], average='micro') for target in targets]

f1_macro_avg = np.mean(f1_scores_macro)
f1_micro_avg = np.mean(f1_scores_micro)

# Print overall results
print("\n---Apply Best Hyper Param for Test Set---")
print(f"Overall Test Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {ham_loss:.4f}")
print(f"F1 Score (Macro): {f1_macro_avg:.4f}")
print(f"F1 Score (Micro): {f1_micro_avg:.4f}")

# AUC scores (assuming auc_scores dictionary is calculated earlier)
print("AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")


---Starting Training Model with SVM---
Best hyperparameters for t2_MAFLD_0: {'C': 10}
Best hyperparameters for t2_MAFLD_Obesity: {'C': 10}
Best hyperparameters for t2_MAFLD_Diabetes: {'C': 10}
Best hyperparameters for t2_MAFLD_MD: {'C': 10}

---Apply Best Hyper Param for Test Set---
Overall Test Accuracy: 0.8584
Hamming Loss: 0.0720
F1 Score (Macro): 0.5500
F1 Score (Micro): 0.9280
AUC Scores for Each Target:
t2_MAFLD_0: 0.8065
t2_MAFLD_Obesity: 0.8275
t2_MAFLD_Diabetes: 0.9261
t2_MAFLD_MD: 0.8243


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, hamming_loss, f1_score
import pandas as pd
import numpy as np

# Initialize DataFrames for storing probabilities and predictions
y_prob_df = pd.DataFrame()
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10]}

print("---Starting Training Model with SVM---")
# Loop through each target variable for model training and evaluation
for target in targets:
    # Initialize SVC and GridSearchCV
    svm = SVC(probability=True)
    grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_svm = grid_search.best_estimator_
    print(f"Best hyperparameters for {target}: {grid_search.best_params_}")

    # Predict probabilities and classes on the test set
    y_prob = best_svm.predict_proba(X_test)[:, 1]
    y_pred = best_svm.predict(X_test)

    # Store the probabilities and predictions
    y_prob_df[target] = y_prob
    y_pred_df[target] = y_pred

    # Calculate and print AUC for the target
    # print("AUC Scores for Each Target:")
    # auc_score = roc_auc_score(y_test[target], y_prob)
    # print(f"{target}: {auc_score:.4f}")

# Calculate overall performance metrics
true_labels_flat = np.hstack([y_test[t] for t in targets])
pred_probs_flat = np.hstack([y_prob_df[t] for t in targets])
pred_labels_flat = np.hstack([y_pred_df[t] for t in targets])

# # After evaluating all models
print("\n---Apply Best Hyper Param for Test Set---")

# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()

# Calculate overall performance metrics
accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
ham_loss = hamming_loss(true_labels_flat, pred_labels_flat)
f1_macro = f1_score(true_labels_flat, pred_labels_flat, average='macro')
f1_micro = f1_score(true_labels_flat, pred_labels_flat, average='micro')

# Print overall results
print(f"Overall Test Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {ham_loss:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
# Print AUC for each target
print("AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")

---Starting Training Model with SVM---
Best hyperparameters for t2_MAFLD_0: {'C': 10}
Best hyperparameters for t2_MAFLD_Obesity: {'C': 10}
Best hyperparameters for t2_MAFLD_Diabetes: {'C': 10}
Best hyperparameters for t2_MAFLD_MD: {'C': 10}

---Apply Best Hyper Param for Test Set---
Overall Test Accuracy: 0.9280
Hamming Loss: 0.0720
F1 Score (Macro): 0.9050
F1 Score (Micro): 0.9280
AUC Scores for Each Target:
t2_MAFLD_0: 0.7967
t2_MAFLD_Obesity: 0.8191
t2_MAFLD_Diabetes: 0.9157
t2_MAFLD_MD: 0.7963


### Decision Tree

In [33]:
# Initialize DataFrames for storing probabilities and predictions
y_prob_df = pd.DataFrame()
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {'max_depth': [None, 10, 20, 30], 
              'min_samples_split': [2, 5, 10], 
              'min_samples_leaf': [1, 2, 4]}

# Loop through each target variable for model training and evaluation
for target in targets:
    # Initialize DecisionTreeClassifier and GridSearchCV
    dt_clf = DecisionTreeClassifier(random_state=2023)
    grid_search = GridSearchCV(dt_clf, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_dt = grid_search.best_estimator_
    print(f"Best hyperparameters for {target}: {grid_search.best_params_}")

    # Predict probabilities and classes on the test set
    y_prob = best_dt.predict_proba(X_test)[:, 1]
    y_pred = best_dt.predict(X_test)

    # Store the probabilities and predictions
    y_prob_df[target] = y_prob
    y_pred_df[target] = y_pred

    # Calculate and print AUC for the target
    # auc_score = roc_auc_score(y_test[target], y_prob)
    # print(f"Decision Tree AUC for {target}: {auc_score:.4f}")

# Calculate overall performance metrics
true_labels_flat = np.hstack([y_test[t] for t in targets])
pred_probs_flat = np.hstack([y_prob_df[t] for t in targets])
pred_labels_flat = np.hstack([y_pred_df[t] for t in targets])

print("\n---Apply Best Hyper Param for Test Set---")

# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()

# Calculate overall performance metrics
accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
ham_loss = hamming_loss(true_labels_flat, pred_labels_flat)
f1_macro = f1_score(true_labels_flat, pred_labels_flat, average='macro')
f1_micro = f1_score(true_labels_flat, pred_labels_flat, average='micro')
auc_micro = roc_auc_score(true_labels_flat, pred_probs_flat)
auc_macro = np.mean([roc_auc_score(y_test[t], y_prob_df[t]) for t in targets])

# Print overall results
print(f"Overall Test Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {ham_loss:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"AUC (Micro): {auc_micro:.4f}")
print(f"AUC (Macro): {auc_macro:.4f}")

# Print AUC for each target
print("AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")


Best hyperparameters for t2_MAFLD_0: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best hyperparameters for t2_MAFLD_Obesity: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best hyperparameters for t2_MAFLD_Diabetes: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best hyperparameters for t2_MAFLD_MD: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5}

---Apply Best Hyper Param for Test Set---
Overall Test Accuracy: 0.9145
Hamming Loss: 0.0855
F1 Score (Macro): 0.8873
F1 Score (Micro): 0.9145
AUC (Micro): 0.9160
AUC (Macro): 0.7038
AUC Scores for Each Target:
t2_MAFLD_0: 0.8065
t2_MAFLD_Obesity: 0.8275
t2_MAFLD_Diabetes: 0.9261
t2_MAFLD_MD: 0.8243


### Emsemble method - Adaboost

In [18]:
# Initialize DataFrames for storing probabilities and predictions
y_prob_df = pd.DataFrame()
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]}

# Loop through each target variable for model training and evaluation
for target in targets:
    # Initialize AdaBoost and GridSearchCV
    ada_clf = AdaBoostClassifier(random_state=2023)
    grid_search = GridSearchCV(ada_clf, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_ada = grid_search.best_estimator_
    print(f"Best hyperparameters for {target}: {grid_search.best_params_}")

    # Predict probabilities and classes on the test set
    y_prob = best_ada.predict_proba(X_test)[:, 1]
    y_pred = best_ada.predict(X_test)

    # Store the probabilities and predictions
    y_prob_df[target] = y_prob
    y_pred_df[target] = y_pred

    # Calculate and print AUC for the target
    auc_score = roc_auc_score(y_test[target], y_prob)
    # print(f"AdaBoost AUC for {target}: {auc_score:.4f}")

# Calculate overall performance metrics
true_labels_flat = np.hstack([y_test[t] for t in targets])
pred_probs_flat = np.hstack([y_prob_df[t] for t in targets])
pred_labels_flat = np.hstack([y_pred_df[t] for t in targets])

accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
auc_micro = roc_auc_score(true_labels_flat, pred_probs_flat)
auc_macro = np.mean([roc_auc_score(y_test[t], y_prob_df[t]) for t in targets])

# Print overall results
print("\n---Apply Best Hyper Param for Test Set---")

# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()

# Calculate overall performance metrics
accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
ham_loss = hamming_loss(true_labels_flat, pred_labels_flat)
f1_macro = f1_score(true_labels_flat, pred_labels_flat, average='macro')
f1_micro = f1_score(true_labels_flat, pred_labels_flat, average='micro')
auc_micro = roc_auc_score(true_labels_flat, pred_probs_flat)
auc_macro = np.mean([roc_auc_score(y_test[t], y_prob_df[t]) for t in targets])

# Print overall results
print(f"Overall Test Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {ham_loss:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"AUC (Micro): {auc_micro:.4f}")
print(f"AUC (Macro): {auc_macro:.4f}")

# Print AUC for each target
print("AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")


Best hyperparameters for t2_MAFLD_0: {'learning_rate': 0.1, 'n_estimators': 100}
Best hyperparameters for t2_MAFLD_Obesity: {'learning_rate': 0.1, 'n_estimators': 100}
Best hyperparameters for t2_MAFLD_Diabetes: {'learning_rate': 0.1, 'n_estimators': 50}
Best hyperparameters for t2_MAFLD_MD: {'learning_rate': 0.01, 'n_estimators': 200}

---Apply Best Hyper Param for Test Set---
Overall Test Accuracy: 0.9371
Hamming Loss: 0.0629
F1 Score (Macro): 0.9168
F1 Score (Micro): 0.9371
AUC (Micro): 0.9707
AUC (Macro): 0.8347
AUC Scores for Each Target:
t2_MAFLD_0: 0.7967
t2_MAFLD_Obesity: 0.8191
t2_MAFLD_Diabetes: 0.9157
t2_MAFLD_MD: 0.7963


### Emsemble method - Bagging

In [19]:
# Initialize DataFrames for storing probabilities and predictions
y_prob_df = pd.DataFrame()
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 1.0], 'max_features': [0.5, 1.0]}

# Loop through each target variable for model training and evaluation
for target in targets:
    # Initialize BaggingClassifier and GridSearchCV
    bagging_clf = BaggingClassifier(random_state=2023)
    grid_search = GridSearchCV(bagging_clf, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_bagging = grid_search.best_estimator_
    print(f"Best hyperparameters for {target}: {grid_search.best_params_}")

    # Predict probabilities and classes on the test set
    y_prob = best_bagging.predict_proba(X_test)[:, 1]
    y_pred = best_bagging.predict(X_test)

    # Store the probabilities and predictions
    y_prob_df[target] = y_prob
    y_pred_df[target] = y_pred

    # Calculate and print AUC for the target
    auc_score = roc_auc_score(y_test[target], y_prob)
    # print(f"Bagging AUC for {target}: {auc_score:.4f}")

# Calculate overall performance metrics
true_labels_flat = np.hstack([y_test[t] for t in targets])
pred_probs_flat = np.hstack([y_prob_df[t] for t in targets])
pred_labels_flat = np.hstack([y_pred_df[t] for t in targets])

accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
auc_micro = roc_auc_score(true_labels_flat, pred_probs_flat)
auc_macro = np.mean([roc_auc_score(y_test[t], y_prob_df[t]) for t in targets])

# Print overall results
print("\n---Apply Best Hyper Param for Test Set---")
# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()

# Calculate overall performance metrics
accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
ham_loss = hamming_loss(true_labels_flat, pred_labels_flat)
f1_macro = f1_score(true_labels_flat, pred_labels_flat, average='macro')
f1_micro = f1_score(true_labels_flat, pred_labels_flat, average='micro')
auc_micro = roc_auc_score(true_labels_flat, pred_probs_flat)
auc_macro = np.mean([roc_auc_score(y_test[t], y_prob_df[t]) for t in targets])

# Print overall results
print(f"Overall Test Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {ham_loss:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"AUC (Micro): {auc_micro:.4f}")
print(f"AUC (Macro): {auc_macro:.4f}")

# Print AUC for each target
print("AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")

Best hyperparameters for t2_MAFLD_0: {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 100}
Best hyperparameters for t2_MAFLD_Obesity: {'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 100}
Best hyperparameters for t2_MAFLD_Diabetes: {'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 100}
Best hyperparameters for t2_MAFLD_MD: {'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 100}

---Apply Best Hyper Param for Test Set---
Overall Test Accuracy: 0.9364
Hamming Loss: 0.0636
F1 Score (Macro): 0.9159
F1 Score (Micro): 0.9364
AUC (Micro): 0.9669
AUC (Macro): 0.7816
AUC Scores for Each Target:
t2_MAFLD_0: 0.7967
t2_MAFLD_Obesity: 0.8191
t2_MAFLD_Diabetes: 0.9157
t2_MAFLD_MD: 0.7963


### Emsemble method - Random Forest

In [35]:
# Initialize DataFrames for storing probabilities and predictions
y_prob_df = pd.DataFrame()
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300], 
    'max_depth': [10, 20, 30], 
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4]
}

# Loop through each target variable for model training and evaluation
for target in targets:
    # Initialize RandomForestClassifier and GridSearchCV
    rf_clf = RandomForestClassifier(random_state=2023)
    grid_search = GridSearchCV(rf_clf, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_rf = grid_search.best_estimator_
    print(f"Best hyperparameters for {target}: {grid_search.best_params_}")

    # Predict probabilities and classes on the test set
    y_prob = best_rf.predict_proba(X_test)[:, 1]
    y_pred = best_rf.predict(X_test)

    # Store the probabilities and predictions
    y_prob_df[target] = y_prob
    y_pred_df[target] = y_pred

    # Calculate and print AUC for the target
    auc_score = roc_auc_score(y_test[target], y_prob)
    # print(f"Random Forest AUC for {target}: {auc_score:.4f}")

# Calculate overall performance metrics
true_labels_flat = np.hstack([y_test[t] for t in targets])
pred_probs_flat = np.hstack([y_prob_df[t] for t in targets])
pred_labels_flat = np.hstack([y_pred_df[t] for t in targets])

# Print overall results
print("---Apply Best Hyper Param for Test Set---")
# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()

# Calculate overall performance metrics
accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
ham_loss = hamming_loss(true_labels_flat, pred_labels_flat)
f1_macro = f1_score(true_labels_flat, pred_labels_flat, average='macro')
f1_micro = f1_score(true_labels_flat, pred_labels_flat, average='micro')
auc_micro = roc_auc_score(true_labels_flat, pred_probs_flat)
auc_macro = np.mean([roc_auc_score(y_test[t], y_prob_df[t]) for t in targets])

# Print overall results
print(f"Overall Test Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {ham_loss:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"AUC (Micro): {auc_micro:.4f}")
print(f"AUC (Macro): {auc_macro:.4f}")

# Print AUC for each target
print("AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")


### Emsemble method - Gradient Boosting

In [37]:
# Initialize DataFrames for storing probabilities and predictions
y_prob_df = pd.DataFrame()
y_pred_df = pd.DataFrame()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Loop through each target variable for model training and evaluation
for target in targets:
    # Initialize GradientBoostingClassifier and GridSearchCV
    gb_clf = GradientBoostingClassifier(random_state=2023)
    grid_search = GridSearchCV(gb_clf, param_grid, cv=5, scoring='roc_auc')

    # Perform hyperparameter tuning
    grid_search.fit(X_train, y_train[target])

    # Best model and parameters after hyperparameter tuning
    best_gb = grid_search.best_estimator_
    print(f"Best hyperparameters for {target}: {grid_search.best_params_}")

    # Predict probabilities and classes on the test set
    y_prob = best_gb.predict_proba(X_test)[:, 1]
    y_pred = best_gb.predict(X_test)

    # Store the probabilities and predictions
    y_prob_df[target] = y_prob
    y_pred_df[target] = y_pred

    # Calculate and print AUC for the target
    auc_score = roc_auc_score(y_test[target], y_prob)
    # print(f"Gradient Boosting AUC for {target}: {auc_score:.4f}")

# Calculate overall performance metrics
true_labels_flat = np.hstack([y_test[t] for t in targets])
pred_probs_flat = np.hstack([y_prob_df[t] for t in targets])
pred_labels_flat = np.hstack([y_pred_df[t] for t in targets])

# Print overall results
print("---Apply Best Hyper Param for Test Set---")
# Convert DataFrame to numpy array for metric calculation
y_pred_array = y_pred_df.to_numpy()

# Calculate overall performance metrics
accuracy = accuracy_score(true_labels_flat, pred_labels_flat)
ham_loss = hamming_loss(true_labels_flat, pred_labels_flat)
f1_macro = f1_score(true_labels_flat, pred_labels_flat, average='macro')
f1_micro = f1_score(true_labels_flat, pred_labels_flat, average='micro')
auc_micro = roc_auc_score(true_labels_flat, pred_probs_flat)
auc_macro = np.mean([roc_auc_score(y_test[t], y_prob_df[t]) for t in targets])

# Print overall results
print(f"Overall Test Accuracy: {accuracy:.4f}")
print(f"Hamming Loss: {ham_loss:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"AUC (Micro): {auc_micro:.4f}")
print(f"AUC (Macro): {auc_macro:.4f}")

# Print AUC for each target
print("AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")

### ANN - 1 hidden layer


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [22]:
# ANN - FC
# convert to PyTorch tensor
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train[targets].values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val[targets].values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test[targets].values, dtype=torch.float32)

# create data loader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [23]:
def calculate_performance(val_loader, model, target_labels):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            predicted = outputs.data
            all_labels.append(labels.cpu().numpy())
            all_predictions.append(predicted.cpu().numpy())

    all_labels = np.vstack(all_labels)
    all_predictions = np.vstack(all_predictions)
    print("Labels shape:", all_labels.shape, "Predictions shape:", all_predictions.shape)

    accuracy = accuracy_score(all_labels, all_predictions > 0.5)
    f1_micro = f1_score(all_labels, all_predictions > 0.5, average='micro')
    f1_macro = f1_score(all_labels, all_predictions > 0.5, average='macro')
    hl = hamming_loss(all_labels, all_predictions > 0.5)

    class_auc = {}
    for i, label in enumerate(target_labels):
        class_auc[f'{label}: AUC'] = roc_auc_score(all_labels[:, i], all_predictions[:, i])

    auc_micro = roc_auc_score(all_labels, all_predictions, average='micro')
    auc_macro = roc_auc_score(all_labels, all_predictions, average='macro')

    return accuracy, f1_micro, f1_macro, hl, auc_micro, auc_macro, class_auc


In [24]:
class SingleLayerNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(SingleLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Adjusted to use hidden_size
        self.fc2 = nn.Linear(hidden_size, output_size) # Adjusted to use hidden_size

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Using sigmoid as the activation function
        return x

In [25]:
# Define input and output size
input_size = X_train.shape[1]
output_size = len(targets)

# Define criterion
criterion = nn.BCELoss()  
epochs = 5

In [26]:
# Hyper param tuning in validation set
hyperparam_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'hidden_size': [16, 32, 64]
}

best_params = None
best_loss = float('inf')

for lr in hyperparam_grid['learning_rate']:
    for hidden_size in hyperparam_grid['hidden_size']:
        model = SingleLayerNN(input_size, output_size, hidden_size)  # Adjust the model to take hidden_size
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        for epoch in range(epochs):
            model.train()
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            # Validation step
            model.eval()
            with torch.no_grad():
                val_loss = 0
                for inputs, labels in val_loader:  # Assuming you have a val_loader
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()
                val_loss /= len(val_loader)
                
            if val_loss < best_loss:
                best_loss = val_loss
                best_params = {'learning_rate': lr, 'hidden_size': hidden_size}

print(f"Best Params: {best_params}, Best Validation Loss: {best_loss}")


Best Params: {'learning_rate': 0.01, 'hidden_size': 16}, Best Validation Loss: 0.19112213887274265


In [27]:
# Reinitialize the model with the best parameters
best_lr = best_params['learning_rate']
best_hidden_size = best_params['hidden_size']
best_model = SingleLayerNN(input_size, output_size, best_hidden_size)

# Retrain the model on the full training dataset
optimizer = optim.Adam(best_model.parameters(), lr=best_lr)
criterion = nn.BCELoss()

for epoch in range(epochs):
    best_model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
best_model.eval()
test_loss = 0
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = best_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        y_pred.extend(outputs.detach().numpy())
        y_true.extend(labels.numpy())

test_loss /= len(test_loader)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
# Convert predictions and true labels to binary format for evaluation
y_pred_binary = np.round(y_pred)
y_true_binary = np.round(y_true)

# Calculate hamming loss
ham_loss = hamming_loss(y_true_binary, y_pred_binary)

# Calculate F1 scores
f1_macro = f1_score(y_true_binary, y_pred_binary, average='macro')
f1_micro = f1_score(y_true_binary, y_pred_binary, average='micro')

accuracy = accuracy_score(np.round(y_true), np.round(y_pred))
auc_scores = {target: roc_auc_score(y_true[:, i], y_pred[:, i]) for i, target in enumerate(targets)}


# Print results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {accuracy}")
print(f"Hamming Loss: {ham_loss}")
print(f"F1 Score (Macro): {f1_macro}")
print(f"F1 Score (Micro): {f1_micro}")
print(f"AUC Scores for Each Target:")
for target, auc in auc_scores.items():
    print(f"{target}: {auc:.4f}")


Test Loss: 0.16842494648881257
Test Accuracy: 0.8830784410458806
Hamming Loss: 0.06216082881105081
F1 Score (Macro): 0.26622769614274466
F1 Score (Micro): 0.8775510204081632
AUC Scores for Each Target:
t2_MAFLD_0: 0.8065
t2_MAFLD_Obesity: 0.8275
t2_MAFLD_Diabetes: 0.9261
t2_MAFLD_MD: 0.8243


###  Proposed - Multitask learning with Autoencoder

In [28]:
class AEwithClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 ==> 4
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(51, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 4)
        )

        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 4 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(4, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 51),
            torch.nn.Sigmoid()
        )

        # classifier
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(4, 4),
            torch.nn.Sigmoid()  # multi label

        )


    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        classification = self.classifier(encoded)
        return decoded, classification

In [29]:
reconstruction_criterion = torch.nn.MSELoss()
classification_criterion = torch.nn.CrossEntropyLoss()

num_epochs = 20

def l1_regularization(model, lambda_param):
    l1_loss = 0
    for param in model.parameters():
        l1_loss += torch.norm(param, p=1)  
    return lambda_param * l1_loss

In [30]:
#Try Different Lambda
lambda_params = [0.01, 0.1, 1, 10]
l1_lambdas = [0.001, 0.01, 0.1]

def train_model(lambda_param, l1_lambda, train_loader, val_loader):
    model = AEwithClassifier()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-8)

    for epoch in range(num_epochs):
        model.train()
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            decoded_outputs, classification_outputs = model(inputs)
            reconstruction_loss = reconstruction_criterion(decoded_outputs, inputs)
            classification_loss = classification_criterion(classification_outputs, labels)
            total_loss = lambda_param * reconstruction_loss + classification_loss + l1_regularization(model, l1_lambda)
            total_loss.backward()
            optimizer.step()

    # Calculate performance on validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            decoded_outputs, classification_outputs = model(inputs)
            reconstruction_loss = reconstruction_criterion(decoded_outputs, inputs)
            classification_loss = classification_criterion(classification_outputs, labels)
            val_loss += lambda_param * reconstruction_loss + classification_loss

    val_loss /= len(val_loader)
    return model, val_loss

# Hyperparameter tuning
best_loss = float('inf')
best_params = None

for lambda_param in lambda_params:
    for l1_lambda in l1_lambdas:
        print(f"Training with lambda param = {lambda_param}, L1 lambda = {l1_lambda}")
        model, val_loss = train_model(lambda_param, l1_lambda, train_loader, val_loader)
        print(f"Validation Loss: {val_loss}")

        if val_loss < best_loss:
            best_loss = val_loss
            best_params = {'lambda_param': lambda_param, 'l1_lambda': l1_lambda}

Training with lambda param = 0.01, L1 lambda = 0.001
Validation Loss: 0.9365230798721313
Training with lambda param = 0.01, L1 lambda = 0.01
Validation Loss: 0.943535566329956
Training with lambda param = 0.01, L1 lambda = 0.1
Validation Loss: 1.2775330543518066
Training with lambda param = 0.1, L1 lambda = 0.001
Validation Loss: 1.0164237022399902
Training with lambda param = 0.1, L1 lambda = 0.01
Validation Loss: 1.037276268005371
Training with lambda param = 0.1, L1 lambda = 0.1
Validation Loss: 1.357501745223999
Training with lambda param = 1, L1 lambda = 0.001
Validation Loss: 1.5315965414047241
Training with lambda param = 1, L1 lambda = 0.01
Validation Loss: 1.8232976198196411
Training with lambda param = 1, L1 lambda = 0.1
Validation Loss: 2.1795289516448975
Training with lambda param = 10, L1 lambda = 0.001
Validation Loss: 6.1094818115234375
Training with lambda param = 10, L1 lambda = 0.01
Validation Loss: 7.371715545654297
Training with lambda param = 10, L1 lambda = 0.1
Va

In [31]:
# Retrain model with best parameters
print(f"Best Parameters: {best_params}")
best_model = train_model(best_params['lambda_param'], best_params['l1_lambda'], train_loader, val_loader)[0]

# Evaluate on test set
test_loss = utils.calculate_performance_AEclassifier(test_loader, best_model)  
# print(f"Test Loss: {test_loss}")

Best Parameters: {'lambda_param': 0.01, 'l1_lambda': 0.001}
Accuracy: 88.55%
F1 Score (Micro): 0.8756
F1 Score (Macro): 0.2348
Hamming Loss: 0.0629
AUC (Micro): 0.9401
AUC (Macro): 0.4270
AUC for t2_MAFLD_0: 0.4641
AUC for t2_MAFLD_Obesity: 0.4693
AUC for t2_MAFLD_Diabetes: 0.3597
AUC for t2_MAFLD_MD: 0.4149
